## データの読み込み
mnistのデータを読み込んで、幾つかデータを見てみましょう。mnist.load_data()でデータをダウンロードし、pythonのオブジェクトとして利用できるようになります。

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

データ件数を確認しておきましょう。

In [ ]:
print(len(training_images), len(test_images))

訓練用データ中のN枚目の画像のサイズや中身を詳細を見てみましょう。

In [ ]:
N=0
print(training_images[N].shape)
print(training_images[N])

N枚目の学習用画像とラベルを表示してみましょう。

In [ ]:
import matplotlib.pyplot as plt
N=0
plt.imshow(training_images[N], cmap='Greys')
plt.show()
print('label=', training_labels[N])

画像データの数値を [0, 1] に正規化しておきましょう。

In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

## NNによる画像認識
いよいよNNのモデルを作成します。3層構造で、中間層の数を5個、出力層は0から9のラベルに合わせて10個とします。

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=5, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

上の例のように空のモデルを作成してから1層ずつ追加しても良いですし、次の例のようにSequential()への引数として3層の情報を渡して作成することもできます。

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(5, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

モデルができたら、compile()によって訓練方法を設定します。

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

いよいよ訓練してみましょう。

In [ ]:
model.fit(training_images, training_labels, epochs=10)

そこそこ良い性能が出たかも知れませんが、大事なのは汎化性能、すなわち訓練には用いていないデータに対して十分な性能を有するかどうかです。そこで、訓練には使用していないテストデータを用いて評価をしてみます。

In [ ]:
model.evaluate(test_images, test_labels)

性能が不十分であれば、モデルを変更したり、訓練方法を見直すなどの手段を講じていきますが、その前に個別の認識結果を確認してみよう。まず、全テストデータの認識結果を保存しておきます。

In [ ]:
classifications = model.predict(test_images)

1枚ずつ認識結果を見てみましょう。softmax層によって出力されるOne hot vectorは、要素の合計が1となるため、各々のラベルの確率を意味していると解釈できます。

In [ ]:
import numpy as np
N=0
print(classifications[N])
print('prediction=', np.argmax(classifications[N]))
print('true label=', test_labels[N])
plt.imshow(test_images[N], cmap='Greys')
plt.show()

訓練したNNの規模を表示してみましょう。訓練の過程で調整したパラメータ数を確認できます。

In [ ]:
model.summary()

## 訓練状況の可視化
訓練の進み具合をグラフ化してみましょう。fit()はepoch毎の評価値を履歴として返してくれるので、これを変数に保存しておけばすぐにグラフ化できます。verbose=0とすることで、途中経過を表示しないようにできます。validation_dataを与えることで、テストデータを用いた検証も同時に行うことができます。

In [ ]:
train_hist=model.fit(training_images, training_labels, epochs=10, verbose=0,
                     validation_data=(test_images, test_labels))

グラフ化します。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
figs, axs=plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(train_hist.history['loss'], label='training')
axs[0].plot(train_hist.history['val_loss'], label='validation')
axs[0].legend()
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[1].plot(train_hist.history['accuracy'], label='training')
axs[1].plot(train_hist.history['val_accuracy'], label='validation')
axs[1].legend()
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
plt.show()